# Импорты

In [1]:
%%time
# здесь будем ставить все, чего нам не хватает
# !pip install missingno -q
# !pip install optuna -q
# !pip install tqdm -q
# !pip install seaborn -q
# !pip install torch -q
# !pip install scikit-learn -q
# !pip install statsmodels -q

CPU times: total: 0 ns
Wall time: 4.05 μs


In [2]:
import pandas as pd
import psycopg2
import logging
from sqlalchemy import create_engine
import sqlalchemy
import time
import socket
import sys

# === Функции проекта ===
from func import (
    check_data,
    drop_duplicated,
    plot_combined,
    calc_target_correlations,
    plot_results,
    plot_categorical_columns,
    plot_scatter_with_numerical
)

from creds import db_config

logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s [%(levelname)s] %(name)s: %(message)s",
    datefmt="%Y-%m-%d %H:%M:%S",
)
logger = logging.getLogger(__name__)

# Подключение к базе и получение данных

In [3]:
# Импортируем конфигурацию из отдельного файла
try:
    logger.info("Конфигурация загружена из creds.py")
except ImportError:
    logger.info("Файл creds.py не найден")
    db_config = {
        'user': input("Введите имя пользователя: "),
        'pwd': input("Введите пароль: "),
        'host': input("Введите хост: "),
        'port': input("Введите порт: "),
        'db': input("Введите название БД: ")
    }

2025-11-24 13:46:40 [INFO] __main__: Конфигурация загружена из creds.py


In [4]:
def check_host_availability():
    """Проверка доступности хоста"""
    try:
        socket.create_connection((db_config['host'], db_config['port']), timeout=10)
        logger.info("Хост доступен")
        return True
    except socket.error as e:
        logger.info(f"Хост недоступен: {e}")
        sys.exit("ОСТАНОВКА: Не удалось подключиться к базе данных. Дальнейшее выполнение невозможно.")

check_host_availability()

2025-11-24 13:46:40 [INFO] __main__: Хост доступен


True

Хост доступен, можем подключаться и загружать данные.   

In [5]:
# если с хостом все ок, то подключаемся и забираем данные
# начнем со всеми данными, потом отфильтруем согласно ТЗ, чтобы облегчить запрос
connection_string = f"postgresql://{db_config['user']}:{db_config['pwd']}@{db_config['host']}:{db_config['port']}/{db_config['db']}"
engine = create_engine(connection_string)

tables_default = {
    'collisions': 'SELECT * FROM collisions',
    'parties': 'SELECT * FROM parties', 
    'vehicles': 'SELECT * FROM vehicles'
}

def load_all_tables(engine, tables):
    """Загружает таблицы из базы данных"""
    
    dataframes = {}
    
    for table_name, query in tables.items():
        try:
            logger.info(f"Загружаем таблицу {table_name}...")
            dataframes[table_name] = pd.read_sql(query, engine)
            logger.info(f"{table_name}: {dataframes[table_name].shape[0]} строк, {dataframes[table_name].shape[1]} столбцов")
        except Exception as e:
            logger.info(f"Ошибка при загрузке {table_name}: {e}")
    
    return dataframes

In [6]:
logger.info("Начинаем загрузку данных из базы...")
df_dict = load_all_tables(engine, tables_default)

# ну и сохраним в отдельные переменные
collisions_default = df_dict['collisions']
parties_default = df_dict['parties']
vehicles_default = df_dict['vehicles']

logger.info("Данные успешно загружены и готовы к анализу.")

2025-11-24 13:46:40 [INFO] __main__: Начинаем загрузку данных из базы...
2025-11-24 13:46:40 [INFO] __main__: Загружаем таблицу collisions...
2025-11-24 13:47:16 [INFO] __main__: collisions: 1400000 строк, 20 столбцов
2025-11-24 13:47:16 [INFO] __main__: Загружаем таблицу parties...
2025-11-24 13:47:45 [INFO] __main__: parties: 2752408 строк, 9 столбцов
2025-11-24 13:47:45 [INFO] __main__: Загружаем таблицу vehicles...
2025-11-24 13:47:53 [INFO] __main__: vehicles: 1021234 строк, 6 столбцов
2025-11-24 13:47:53 [INFO] __main__: Данные успешно загружены и готовы к анализу.


In [7]:
collisions_default.head()

,case_id,county_city_location,county_location,distance,direction,intersection,weather_1,location_type,collision_damage,party_count,primary_collision_factor,pcf_violation_category,type_of_collision,motor_vehicle_involved_with,road_surface,road_condition_1,lighting,control_device,collision_date,collision_time
0,4083072,1942,los angeles,528.0,north,0.0,cloudy,highway,small damage,2,vehicle code violation,unsafe lane change,sideswipe,other motor vehicle,wet,normal,daylight,none,2009-01-22,07:25:00
1,4083075,4313,santa clara,0.0,None,1.0,clear,None,small damage,1,vehicle code violation,improper passing,hit object,fixed object,dry,normal,dark with street lights,functioning,2009-01-03,02:26:00
2,4083073,0109,alameda,0.0,None,1.0,clear,None,scratch,2,vehicle code violation,improper turning,broadside,other motor vehicle,dry,normal,dark with street lights,functioning,2009-01-11,03:32:00
3,4083077,0109,alameda,0.0,None,1.0,clear,None,scratch,2,vehicle code violation,automobile right of way,broadside,other motor vehicle,dry,normal,daylight,functioning,2009-01-11,10:35:00
4,4083087,4313,santa clara,0.0,None,1.0,clear,None,scratch,2,vehicle code violation,speeding,rear end,other motor vehicle,dry,None,dark with street lights,functioning,2009-01-02,22:43:00


In [8]:
parties_default.head()

,id,case_id,party_number,party_type,at_fault,insurance_premium,party_sobriety,party_drug_physical,cellphone_in_use
0,22,3899454,1,road signs,1,29.0,had not been drinking,None,0.0
1,23,3899454,2,road signs,0,7.0,had not been drinking,None,0.0
2,29,3899462,2,car,0,21.0,had not been drinking,None,0.0
3,31,3899465,2,road signs,0,24.0,had not been drinking,None,0.0
4,41,3899478,2,road bumper,0,NaN,not applicable,not applicable,0.0


In [9]:
vehicles_default.head()

,id,case_id,party_number,vehicle_type,vehicle_transmission,vehicle_age
0,1175713,5305032,2,sedan,manual,3.0
1,1,3858022,1,sedan,auto,3.0
2,1175712,5305030,1,sedan,auto,3.0
3,1175717,5305033,3,sedan,auto,5.0
4,1175722,5305034,2,sedan,auto,5.0


Да, много данных. Посмотрим, как это число изменится если мы сразу получим только то, что нам нужно

В тз у нас есть ограничения на выборку - учтем их сразу в запросах, чтобы не тащить все данные.  
- Для модели выбрать тип виновника — только машина (car).  
- Выбрать случаи, когда ДТП привело к любым повреждениям транспортного средства, кроме типа SCRATCH (царапина).  
- Для моделирования ограничиться данными за 2012 год — они самые свежие. 

# Отбор признаков для модели предсказания виновности в ДТП

## Признаки для модели

Из таблицы `collisions`

| Признак | Обоснование |
|---------|-------------|
| `WEATHER_1` | Погодные условия напрямую влияют на видимость и управляемость транспортным средством |
| `ROAD_SURFACE` | Состояние дорожного покрытия значительно воздействует на риск возникновения ДТП |
| `LIGHTING` | Уровень освещённости влияет на видимость и скорость реакции водителя |
| `INTERSECTION` | Перекрестки являются зонами повышенной аварийности и сложности манёвров |
| `LOCATION_TYPE` | Тип дороги определяет характер движения и потенциальные риски |
| `PCF_VIOLATION_CATEGORY` | Ключевой признак - категории нарушений ПДД напрямую коррелируют с виновностью |
| `TYPE_OF_COLLISION` | Тип столкновения может указывать на действия виновника аварии |
| `ROAD_CONDITION_1` | Дополнительные дорожные условия создают дополнительные факторы риска |

Из таблицы `parties`

| Признак | Обоснование |
|---------|-------------|
| `PARTY_DRUG_PHYSICAL` | Важный признак - физическое состояние водителя напрямую влияет на виновность |
| `PARTY_SOBRIETY` | Критический признак - состояние трезвости является ключевым фактором определения виновника |
| `CELLPHONE_IN_USE` | Использование телефона за рулём приводит к отвлечению внимания и нарушениям |

Из таблицы `vehicles`

| Признак | Обоснование |
|---------|-------------|
| `VEHICLE_AGE` | Обязателен по ТЗ - возраст автомобиля может влиять на техническую исправность и безопасность |
| `VEHICLE_TYPE` | Тип кузова может влиять на управляемость и поведение транспортного средства |

---

## Признаки для исключения из модели

Технические и идентификационные признаки
- `CASE_ID`, `ID` - уникальные идентификаторы, не несут прогнозной способности
- `PARTY_NUMBER` - технический номер участника в рамках ДТП

Географические признаки
- `COUNTY_CITY_LOCATION`, `COUNTY_LOCATION` - без детальных карт и анализа дорожной инфраструктуры сложно интерпретировать
- `DIRECTION` - направление движения само по себе не является информативным признаком

Временные признаки
- `COLLISION_DATE`, `COLLISION_TIME` - рекомендуется вынести в отдельный анализ сезонности и времени суток

Избыточные и производные признаки
- `PRIMARY_COLLISION_FACTOR` - дублирует информацию из `PCF_VIOLATION_CATEGORY`, но в более обобщённой форме
- `PARTY_COUNT` - количество участников ДТП слабо коррелирует с виновностью конкретного водителя
- `DISTANCE` - расстояние от главной дороги без дополнительного контекста не информативно
- `MOTOR_VEHICLE_INVOLVED_WITH` - тип второго участника менее важен, чем действия самого виновника
- `INSURANCE_PREMIUM` - страховая премия является скорее следствием, чем причиной ДТП

In [12]:
# оптимизированные запросы с учетом отбора признаков
collisions_query = """
SELECT 
    case_id,
    weather_1,
    road_surface,
    lighting,
    intersection,
    location_type,
    pcf_violation_category,
    type_of_collision,
    road_condition_1
FROM collisions
WHERE EXTRACT(YEAR FROM collision_date) = 2012
    AND collision_damage <> 'scratch'
"""

parties_query = """
SELECT 
    case_id,
    party_number,
    at_fault,
    party_drug_physical,
    party_sobriety,
    cellphone_in_use
FROM parties 
WHERE party_type = 'car'
"""

vehicles_query = """
SELECT 
    case_id,
    party_number,
    vehicle_type,
    vehicle_age,
    vehicle_transmission
FROM vehicles
WHERE case_id IN (
    SELECT DISTINCT case_id 
    FROM parties 
    WHERE party_type = 'car'
)
"""

tables_clear = {
    'collisions': collisions_query,
    'parties': parties_query, 
    'vehicles': vehicles_query
}

In [13]:
logger.info("Начинаем загрузку данных из базы...")
df_dict = load_all_tables(engine, tables_clear)

# ну и сохраним в отдельные переменные
collisions = df_dict['collisions']
parties = df_dict['parties']
vehicles = df_dict['vehicles']

logger.info("Данные успешно загружены и готовы к анализу.")

2025-11-24 13:48:42 [INFO] __main__: Начинаем загрузку данных из базы...
2025-11-24 13:48:42 [INFO] __main__: Загружаем таблицу collisions...
2025-11-24 13:48:44 [INFO] __main__: collisions: 115666 строк, 9 столбцов
2025-11-24 13:48:44 [INFO] __main__: Загружаем таблицу parties...
2025-11-24 13:49:04 [INFO] __main__: parties: 2469314 строк, 6 столбцов
2025-11-24 13:49:04 [INFO] __main__: Загружаем таблицу vehicles...
2025-11-24 13:49:14 [INFO] __main__: vehicles: 1013759 строк, 5 столбцов
2025-11-24 13:49:14 [INFO] __main__: Данные успешно загружены и готовы к анализу.


# Сравнительная таблица оптимизации данных

## Эффективность очистки данных

| Параметр | Было | Стало | Экономия |
|----------|------|-------|----------|
| Общее время загрузки | ~73 секунды | ~32 секунды | 56% быстрее |

Таблица `collisions`

| Показатель | Было | Стало | Изменение |
|------------|------|-------|-----------|
| Количество строк | 1,400,000 | 115,666 | 92% меньше |
| Количество столбцов | 20 | 9 | 55% меньше |
| Время загрузки | ~36 сек | ~2 сек | 94% быстрее |

Убраны столбцы: `collision_date`, `collision_time`, `county_city_location`, `county_location`, `direction`, `distance`, `party_count`, `primary_collision_factor`, `motor_vehicle_involved_with`, `collision_damage`

Таблица `parties`

| Показатель | Было | Стало | Изменение |
|------------|------|-------|-----------|
| Количество строк | 2,752,408 | 2,469,314 | 10% меньше |
| Количество столбцов | 9 | 6 | 33% меньше |
| Время загрузки | ~29 сек | ~20 сек | 31% быстрее |

Убраны столбцы: `insurance_premium`, `id` (технические)

Таблица `vehicles`

| Показатель | Было | Стало | Изменение |
|------------|------|-------|-----------|
| Количество строк | 1,021,234 | 1,013,759 | 1% меньше |
| Количество столбцов | 6 | 5 | 17% меньше |
| Время загрузки | ~8 сек | ~10 сек | 25% медленнее |

Убран столбец: `id` (технический)

## Ключевые улучшения

Производительность
- Общее время загрузки сокращено на 41 секунду
- Наибольший выигрыш в таблице collisions - с 36 до 2 секунд
- Оптимизированные запросы с фильтрацией на стороне БД

Качество данных
- Сфокусированы на значимых признаках для модели
- Убраны технические и избыточные поля
- Сохранены все ключевые переменные для анализа

Эффективность памяти
- Уменьшен объем хранимых данных
- Улучшена скорость обработки в дальнейших этапах
- Сохранена информативность для построения модели

Итоговый выигрыш

> Оптимизация данных позволила ускорить загрузку в 2.3 раза, сохранив при этом все необходимые для анализа признаки и улучшив качество данных для построения модели машинного обучения.

# Проведите первичное исследование таблиц